## Exercise 2: Bias-variance trade-off and resampling techniques 

#### Overview of nature of the problem: 



In [1]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
import common
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
#create data: 
# np.random.seed(4155)
# n = 11
# N = 200
# x = np.sort(np.random.uniform(0, 1, N))
# y = np.sort(np.random.uniform(0, 1, N))
# z = common.FrankeFunction(x, y) + 0.2*np.random.rand(N)



n = 40
n_boostraps = 100
maxdegree = 14

# Make data set.
x = np.linspace(-3, 3, n).reshape(-1, 1)
y = np.exp(-x**2) + 1.5 * np.exp(-(x-2)**2)+ np.random.normal(0, 0.1, x.shape)

In [3]:
#TODO: Number of bootstraps? 
n_boostraps = 100
MSE_test = np.zeros(n)
MSE_train = np.zeros(n)
polydegree = np.zeros(n)
bias = np.zeros(n)
variance = np.zeros(n)
#for each degree of poly:


for degree in tqdm(range(1,maxdegree), desc = f"Looping through polynomials up to {n} degrees with {n_boostraps} bootstraps: "):
    #model = LinearRegression()
    model= make_pipeline(LinearRegression(fit_intercept=False)) 
    
    #X = common.create_X(x, y, n=degree)   
    #print(f"at pol.degree: {degree}, X has shape: {X.shape}") 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

    #reshape for broadcasting in MSE_test and MSE_val  
    y_test_ = np.reshape(y_test, newshape=(y_test.shape[0],1))
    #reshape for broadcasting in MSE_test and MSE_val  
    y_train_ = np.reshape(y_train, newshape=(y_train.shape[0],1))

    #y_train = np.reshape(y_train, newshape=(y_train.shape[0],1))
    #TODO: why scale?!?


    y_pred = np.empty((y_test.shape[0], n_boostraps))
    y_fit = np.empty((y_train.shape[0], n_boostraps))
    
    for i in range(n_boostraps):
        #bootstrap:
        x_, y_ = resample(X_train, y_train)
        #fit model to x_,y_ sample:
        #print(f"y_.shape : {y_.shape}")
        clf =  model.fit(x_, y_)
        #fit model and predict on test data:
        y_pred[:, i] = clf.predict(X_test).ravel()
        #predict on train data:
        y_fit[:,i] = clf.predict(X_train).ravel()
        

    polydegree[degree] = degree
   
    MSE_test[degree] = np.mean( np.mean((y_test_ - y_pred)**2, axis=1, keepdims=True) )
    MSE_train[degree] = np.mean( np.mean((y_train_ - y_fit)**2, axis=1, keepdims=True) )
    bias[degree] = np.mean( (y_test - np.mean(y_pred, axis=1, keepdims=True))**2 )
    variance[degree] = np.mean( np.var(y_pred, axis=1, keepdims=True))
    
plt.plot(polydegree, MSE_test, label='MSE_test')
plt.plot(polydegree, MSE_train, label='MSE_train')


plt.xlabel("Model complexity / Polynomial Degree")
plt.ylabel("Prediction Error")
plt.legend()
plt.show()

plt.plot(polydegree, bias, label='bias')
plt.plot(polydegree, variance, label='Variance')


plt.legend()
plt.show()




Looping through polynomials up to 40 degrees with 100 bootstraps:   0%|          | 0/13 [00:00<?, ?it/s]


NameError: name 'X_train_scaled' is not defined